# Clustering

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))
%load_ext autoreload
%autoreload 2

In [2]:
import sys

if ('..' not in sys.path):
    print('append .. to sys.path')
    sys.path.append('..')
print(sys.path)

import numpy as np
import scipy.sparse
# import sparsechem as sc
import argparse
import leader_follower as lf

append .. to sys.path
['/home/kbardool/WSL-projs/chembl_pipeline', '/home/kbardool/miniconda3/envs/pyt-gpu/lib/python39.zip', '/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9', '/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/lib-dynload', '', '/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/site-packages', '/home/kbardool/WSL-projs/leader_follower', '/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/site-packages/IPython/extensions', '/home/kbardool/.ipython', '..']


In [3]:
for i in sys.path:
    print(i)

/home/kbardool/WSL-projs/chembl_pipeline
/home/kbardool/miniconda3/envs/pyt-gpu/lib/python39.zip
/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9
/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/lib-dynload

/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/site-packages
/home/kbardool/WSL-projs/leader_follower
/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/site-packages/IPython/extensions
/home/kbardool/.ipython
..


In [4]:
def print_dbg(text, verbose = False):
    if verbose:
        print(text)

In [5]:
def load_sparse(filename):
    """Loads sparse from Matrix market or Numpy .npy file."""
    if filename is None:
        return None
    if filename.endswith('.mtx'):
        return scipy.io.mmread(filename).tocsr()
    elif filename.endswith('.npy'):
        return np.load(filename, allow_pickle=True).item().tocsr()
    elif filename.endswith('.npz'):
        return scipy.sparse.load_npz(filename).tocsr()
    raise ValueError(f"Loading '{filename}' failed. It must have a suffix '.mtx', '.npy', '.npz'.")


In [6]:
def hierarchical_clustering(X, dists, verbose = False):
    """
    Args:
        X       compound matrix in CSR
        dists   list of (increasing) distances
    Sequentially clusters with each dists, returns final cluster ids
    """
    assert type(X) == scipy.sparse.csr.csr_matrix, "X should be csr_matrix (scipy.sparse)"
    
    print(f"X shape: {X.shape}")
    cl0 = np.arange(X.shape[0])
    Xc  = X

    for dist in dists:
        print(f"Running clustering for distance: {dist}")
        cl, cent = cluster(Xc, dist, verbose)
        Xc       = Xc[cent]
        cl0      = cl[cl0]

    return cl0


## main

In [7]:
parser = argparse.ArgumentParser(description="Training a multi-task model.")
parser.add_argument("--x"  , help="Descriptor file (matrix market or numpy)", type=str, required=True)
parser.add_argument("--out", help="Output file for the clusters (.npy)", type=str, required=True)
parser.add_argument("--dists", nargs="+", help="Distances", default=[], type=float, required=True)


input_args =  "python ../leader_follower/cluster.py"\
              " --x      output/chembl_29_dev/chembl_29_dev_X.npy " \
              " --out    output/chembl_29_dev/chembl_29_dev_clustering.npy "\
              " --dist   0.5 0.7".split()
print(input_args)

args = parser.parse_args(input_args[2:])
print(vars(args))
dists = args.dists

['python', '../leader_follower/cluster.py', '--x', 'output/chembl_29_dev/chembl_29_dev_X.npy', '--out', 'output/chembl_29_dev/chembl_29_dev_clustering.npy', '--dist', '0.5', '0.7']
{'x': 'output/chembl_29_dev/chembl_29_dev_X.npy', 'out': 'output/chembl_29_dev/chembl_29_dev_clustering.npy', 'dists': [0.5, 0.7]}


In [8]:
print(f"Loading '{args.x}'.")
# X = sc.load_sparse(args.x).tocsr()
X = load_sparse(args.x).tocsr()

Loading 'output/chembl_29_dev/chembl_29_dev_X.npy'.


In [9]:
print(X.shape)
print(repr(X))
print(X.__dict__)
for i in X.__dict__:
    print(f" {i:30s} :    {X.__dict__[i]}")
# Xnorm = np.array((X != 0).sum(axis = 1)).flatten()
# Xnorm.shape
print((X.data == 1.0).all())

(423737, 32000)
<423737x32000 sparse matrix of type '<class 'numpy.float64'>'
	with 32838752 stored elements in Compressed Sparse Row format>
{'_shape': (423737, 32000), 'maxprint': 50, 'indices': array([  349,   520,  1482, ..., 31109, 31834, 31968], dtype=int32), 'indptr': array([       0,       79,      162, ..., 32838606, 32838684, 32838752],
      dtype=int32), 'data': array([1., 1., 1., ..., 1., 1., 1.]), '_has_canonical_format': True, '_has_sorted_indices': True}
 _shape                         :    (423737, 32000)
 maxprint                       :    50
 indices                        :    [  349   520  1482 ... 31109 31834 31968]
 indptr                         :    [       0       79      162 ... 32838606 32838684 32838752]
 data                           :    [1. 1. 1. ... 1. 1. 1.]
 _has_canonical_format          :    True
 _has_sorted_indices            :    True
True


## `cluster` code 

In [10]:
%load_ext Cython
# del cluster

In [11]:
# del cluster

In [12]:
%%cython --cplus --force

from libcpp.vector cimport vector
import numpy as np
np.set_printoptions(linewidth = 132)
def cluster(X, radius, verbose1 = False, verbose2 = False):
    
    cdef int i, j, k, row, col, closest
    cdef double min_dist, tmp_dist
    cdef int num_changed
    
    cdef int tmp_i = 10
    cdef int test_limit = 5
    
    cdef int [:] x_indptr  = X.indptr
    cdef int [:] x_indices = X.indices
    cdef double [:] x_data = X.data

    cdef int X_shape0 = X.shape[0]
    cdef int X_shape1 = X.shape[1]


    cdef vector[double]      dist
    cdef vector[vector[int]] centers
    cdef vector[double]      Cnorm
    cdef vector[int]         center_ids

    ## Xnorm[]: no of non-zero columns in each row
    cdef long   [:] Xnorm = np.array((X != 0).sum(axis = 1)).flatten()
    
    ## row_dists[]:
    cdef double [:] row_dists = np.zeros(X_shape0, dtype=np.float64)

    ## Clusters[] : cluster assignment of each X row
    cdef long [:] clusters = np.zeros(X.shape[0], dtype = np.int64) - 1
    
    ## perm_ids[] : Random permutation of row numbers to provide randomize selection of rows 
    cdef long [:] perm_ids = np.random.permutation(X.shape[0])       

    
    print( f"X shape        :      {X.shape}")
    print( f"X.indptr  shape:      {X.indptr.shape}")
    print( f"X.indices shape:      {X.indices.shape}")    
    print( f"X.data    shape:      {X.data.shape}")    
#     print(f" ==== perm ids:       {np.asarray(perm_ids)[0:100]}")

    print(f"\n ==== perm_ids - randomized list of rows ")
    print(f" ==== perm_ids size:   {perm_ids.shape}")
    print(f" ==== perm_ids[:{tmp_i}]:   {np.asarray(perm_ids)[:tmp_i]}. . . . .")


    print(f"\n ==== Xnorm[x] - Norm of row id <x> == number of non-zero columns in row <x>")
    print(f" ==== Xnorm shape:     {Xnorm.shape}")
    print(f" ==== Xnorm [:{tmp_i}]:     {np.asarray(Xnorm)[:tmp_i]} . . . . ." )

    print()
    print(f"\n ==== Cnorm[x] - Norm of cluster <x> centroid ")
    print(f" ==== Cnorm size :     {Cnorm.size()}")  
    print(f" ==== Cnorm[:{tmp_i}]:     {np.asarray(Cnorm)[:tmp_i]} . . . . .") 
    
    print(f"\n ==== row_dists - distance of current row to centroids")
    print(f" ==== row_dists size     :      {row_dists.shape}")
    print(f" ==== row_dists[:{tmp_i}]:      {np.asarray(row_dists)[:tmp_i]} . . . . .")

    print(f"\n ==== dist - ???")
    print(f" ==== dist size:       {dist.size()}")
    print(f" ==== dist[:{tmp_i}]:       {np.asarray(dist)[:tmp_i]} . . . . .")    
    
    print(f"\n ==== centers - ???")
    print(f" ==== centers size     {centers.size()}")
    print(f" ==== centers[:{tmp_i}]:    {np.asarray(centers)[:tmp_i]} . . . . .")     
    centers.resize(X_shape1)
    print(f" ==== centers size     {centers.size()}")
    print(f" ==== centers[:{tmp_i}]:    {np.asarray(centers)[:tmp_i]} . . . . .") 
    

    print()
    print(f"\n ==== center_ids[i] - row # corresponding to centriod of cluster <i> ")    
    print(f" ==== center_ids size: {center_ids.size()}")    
    print(f" ==== center_ids[:{tmp_i}]: {np.asarray(center_ids)[:tmp_i]} . . . . .")    
    print()    
    
    print(f"\n ==== clusters[x] - cluster row x is assigned to ")
    print(f" ==== clusters size:   {clusters.shape}")
    print(f" ==== clusters[:{tmp_i}]:   {np.asarray(clusters)[:tmp_i]} . . . . .")
    print()    
    


    ## algorithm part

    print("=====================================")
    print(" Creating initial Cluster Centroids  ")
    print("=====================================")  
    
    for i in range(X_shape0):
 
        if i % 10000 == 0:
            print(f"Row {i}.")
                
        row = perm_ids[i]
        
        ## Initialize list of distances to all centroids to zero
        row_dists[0 : Cnorm.size()] = 0.0
        
#         if verbose1:
#             print(f"\n    1 - {i:2d}/{X_shape0} - row = perm_ids[i]: {perm_ids[i]:6d}  - x_indptrs: "
#                   f"{x_indptr[row]:9d}  ~ {x_indptr[row+1]:9d}   #elems: {x_indptr[row+1]-x_indptr[row]:9d}   size(Cnorm): {Cnorm.size()}  Xnorm[row]: {Xnorm[row]}")
                        
        ## for current row, get column indices and data of populated columns 
        for j in range(x_indptr[row], x_indptr[row+1]):
            # get column number
            col = x_indices[j]
#             if verbose1:
#                 print(f"      1.1 - j: {j-x_indptr[row] :4d}   row: {row:8d}   col: {col:5d}  data: {x_data[j]}    k= size(centers[{col:5d}]): {centers[col].size()}")
                
            for k in range(centers[col].size()):
#                 if verbose1:
#                     print(f"      1.2  k:  {k:4d}    set row_dists[centers[{col:4d}][{k:4d}]] = row_dists[{centers[col][k]}] = {row_dists[centers[col][k]]} += 1 ->  {row_dists[centers[col][k]]+1}")
                row_dists[ centers[col][k] ] += 1.0
                
#         if verbose1:
#             print(f"      1.3  i: {i:2d}/{X_shape0} - row = {perm_ids[i]:6d}  elems: {x_indptr[row+1]-x_indptr[row]:9d}   size(Cnorm): {Cnorm.size()}")
                
        closest = -1
        min_dist = radius
        
        ## Calculate Distance from Centroids 
#         if verbose1:
#             print(f"\n    2 - Calculate distance from {Cnorm.size()} centroids")
            
        for j in range(Cnorm.size()):
#             if verbose1:
#                 print(f"      2.1 - j/ cluster id: {j}   Xnorm[{row}]: {Xnorm[row]} -  row_dists[{j}] = 1.0 - {row_dists[j]} / ({Xnorm[row]} + {Cnorm[j]} - {row_dists[j]})"\
#                       f" = { 1.0 - row_dists[j] / (Xnorm[row] + Cnorm[j] - row_dists[j]):.4f}")            
            row_dists[j] = 1.0 - row_dists[j] / (Xnorm[row] + Cnorm[j] - row_dists[j])
            if row_dists[j] < min_dist:
#                 if verbose1:
#                     print(f"      2.2 - {row_dists[j]} < {min_dist} - closest set to {j} ")                         
                min_dist = row_dists[j]
                closest  = j
        
        # If a centroid was found within min_distance, assign row to that centroid
        # and move on to next row
        if closest >= 0:
            clusters[row] = closest
#             if verbose1:
#                 print(f"      2.3 - assign row {row} to cluster : {closest} ") 
            continue

        ## Otherwise, create a new cluster
#         if verbose1:
#             print(f"\n    3 - Assign Create a new cluster k: {Cnorm.size()}  (Cnorm.size())   Norm of centroid (Xnorm[row]) : {Xnorm[row]}")
  
        k = Cnorm.size()
        
        for j in range(x_indptr[row], x_indptr[row+1]):
            # Add new element at end of centers
            # centers[ x_indices[j] ].push_back(k)
            col = x_indices[j]
            centers[col].push_back(k)
#             if verbose1:
#                 print(f"      3.1 - j: {j-x_indptr[row]:4d}  - :  centers[{x_indices[j]:5d}] append k = {k:3d} "\
#                   f" to centers[{x_indices[j]:5d}]  -  length(centers[{x_indices[j]:5d}]) is now {centers[ x_indices[j]].size():5d}")
                
        clusters[row] = k
        Cnorm.push_back(Xnorm[row])
        center_ids.push_back(row)
#         if verbose1:
#             print(f"      3.2 - clusters[{row}] = {k}     Cnorm.append(norm of row{row})  ({Xnorm[row]}) - len(Cnorm):{Cnorm.size()}  center_ids.append({row}) - len(center_ids): {center_ids.size()})")
#             print("      ", Cnorm)
#             print(f"       center_ids.add(row: {row})    new size(center_ids):{center_ids.size()}")
#             print("      ", center_ids)
    
    
    print("=====================================")
    print(f" Cluster Centroids Created - {len(center_ids)} clusters.")
    print("=====================================")  
        
    print(f"\n ==== perm_ids - randomized list of rows ")
    print(f" ==== perm_ids size:   {perm_ids.shape}")
    print(f" ==== perm_ids[:{tmp_i}]:   {np.asarray(perm_ids)[:tmp_i]}. . . . .")


    print(f"\n ==== Xnorm - number of non-zero columns in each row")
    print(f" ==== Xnorm shape:     {Xnorm.shape}")
    print(f" ==== Xnorm [:{tmp_i}]:     {np.asarray(Xnorm)[:tmp_i]} . . . . ." )
    
    print(f"\n ==== row_dists - distance of current row to centroids")
    print(f" ==== row_dists size     :      {row_dists.shape}")
    print(f" ==== row_dists[:{tmp_i}]:      {np.asarray(row_dists)[:tmp_i]} . . . . .")
    print()

    print(f"\n ==== dist - ???")
    print(f" ==== dist size:       {dist.size()}")
    print(f" ==== dist[:{tmp_i}]:       {np.asarray(dist)[:tmp_i]} . . . . .")    
    print()
    
    print(f"\n ==== centers ")
    print(f" ==== centers size     {centers.size()}")
#     print(f" ==== centers[:{tmp_i}]:    {np.asarray(centers)[:tmp_i]} . . . . .")     

    
    print()
    print(f"\n ==== Cnorm - ???")
    print(f" ==== Cnorm size :     {Cnorm.size()}")  
    print(f" ==== Cnorm[:{tmp_i}]:     {np.asarray(Cnorm)[:tmp_i]} . . . . .") 
    print()
    print(f"\n ==== center_ids - list of centroid row ids")    
    print(f" ==== center_ids size: {center_ids.size()}")    
    print(f" ==== center_ids[:{tmp_i}]: {np.asarray(center_ids)[:tmp_i]} . . . . .")    
    print()    
    
    print(f"\n ==== clusters - ???")
    print(f" ==== clusters size:   {clusters.shape}")
    print(f" ==== clusters[:{tmp_i}]:   {np.asarray(clusters)[:tmp_i]} . . . . .")
    print()    
    
#     return np.asarray(clusters), np.asarray(center_ids)
    
    print("==============================================")
    print("Reassigning compounds to the closest clusters.")
    print("==============================================") 
    num_changed = 0
    
    for row in range(X_shape0):

        if row % 50000 == 0:
            print(f"Row {row}.")
        
        row_dists[0 : Cnorm.size()] = 0.0
        ## compute distances to all clusters, assign to the closest
        for j in range(x_indptr[row], x_indptr[row+1]):
            col = x_indices[j]
            for k in range(centers[col].size()):
                row_dists[ centers[col][k] ] += 1.0

        closest = -1
        min_dist = radius + 1e-5
        
        # iterate over centroids, calculate distance to Clusters
        for j in range(Cnorm.size()):
            try:
                tmp_dist = 1.0 - row_dists[j] / (Xnorm[row] + Cnorm[j] - row_dists[j])
            except ZeroDivisionError:
                print(f" Zero Division Error Encountered  - row:{row}   j: {j}"\
                      f" Xnorm[{row}]:   {Xnorm[row]}    Cnorm[{j}]:   {Cnorm[j]}      row_dists[{j}]:   {row_dists[j]}")
            else:
                if tmp_dist < min_dist:
                    min_dist = tmp_dist
                    closest  = j
                    if min_dist == 0:
                        ## best possible found, distance is zero
                        break
                        
        if (closest >= 0) and (clusters[row] != closest):
            clusters[row]  = closest
            num_changed   += 1

    print(f"Reassignement changed {num_changed} assignments.")
    print(f"Total {len(center_ids)} clusters.")

    return np.asarray(clusters), np.asarray(center_ids)

/home/kbardool/.cache/ipython/cython/_cython_magic_d6a5b9e7ae5cee4346f96c4bde2375e1.cpp: In function ‘PyObject* __pyx_pf_46_cython_magic_d6a5b9e7ae5cee4346f96c4bde2375e1_cluster(PyObject*, PyObject*, PyObject*, PyObject*, PyObject*)’:
/home/kbardool/.cache/ipython/cython/_cython_magic_d6a5b9e7ae5cee4346f96c4bde2375e1.cpp:4245:39: warning: comparison of integer expressions of different signedness: ‘int’ and ‘std::vector<int>::size_type’ {aka ‘long unsigned int’} [-Wsign-compare]
 4245 |       for (__pyx_t_21 = 0; __pyx_t_21 < __pyx_t_23; __pyx_t_21+=1) {
      |                            ~~~~~~~~~~~^~~~~~~~~~~~
/home/kbardool/.cache/ipython/cython/_cython_magic_d6a5b9e7ae5cee4346f96c4bde2375e1.cpp:4297:37: warning: comparison of integer expressions of different signedness: ‘int’ and ‘std::vector<double>::size_type’ {aka ‘long unsigned int’} [-Wsign-compare]
 4297 |     for (__pyx_t_17 = 0; __pyx_t_17 < __pyx_t_27; __pyx_t_17+=1) {
      |                          ~~~~~~~~~~~^~~~~~~~~~~

## Hierarchical Clustering

In [13]:
import os 
os.getcwd()

'/home/kbardool/WSL-projs/chembl_pipeline'

In [ ]:
print("Clustering.")
# two step clustering, first at 0.5, then at 0.6
cl_hier = hierarchical_clustering(X, dists=args.dists, verbose = True)
# cl, cent = cluster(X, 0.5 )

Clustering.
X shape: (423737, 32000)
Running clustering for distance: 0.5
X shape        :      (423737, 32000)
X.indptr  shape:      (423738,)
X.indices shape:      (32838752,)
X.data    shape:      (32838752,)

 ==== perm_ids - randomized list of rows 
 ==== perm_ids size:   [423737, 0, 0, 0, 0, 0, 0, 0]
 ==== perm_ids[:10]:   [ 62136 266478 335073 416630 109178 388027 398708 325484 261843  64512]. . . . .

 ==== Xnorm[x] - Norm of row id <x> == number of non-zero columns in row <x>
 ==== Xnorm shape:     [423737, 0, 0, 0, 0, 0, 0, 0]
 ==== Xnorm [:10]:     [ 79  83  87 185  49 167 127  78  81  97] . . . . .


 ==== Cnorm[x] - Norm of cluster <x> centroid 
 ==== Cnorm size :     0
 ==== Cnorm[:10]:     [] . . . . .

 ==== row_dists - distance of current row to centroids
 ==== row_dists size     :      [423737, 0, 0, 0, 0, 0, 0, 0]
 ==== row_dists[:10]:      [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] . . . . .

 ==== dist - ???
 ==== dist size:       0
 ==== dist[:10]:       [] . . . . .

 ==== 

In [19]:
print(type(cl_hier), cl_hier.shape)
# cl_hier.min()

NameError: name 'cl_hier' is not defined

In [53]:
# tmp =  np.array((X != 0).sum(axis=1))
# print(tmp.shape)
# tmp = tmp.flatten()
# print(tmp.shape)
# print(type(tmp))
# print(tmp[1000:1200])
# del tmp


    /home/kbardool/WSL-projs/chembl-pipeline-original/../leader_follower/cluster.py:36: 
    DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. 
    To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. 
    When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish
    to review your current use, check the release note link for additional information.
    Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
      cl_hier = leader_follower.hierarchical_clustering(X, dists=args.dists)
    Row 0.


In [14]:
np.save(args.out, cl_hier)
print(f"Saved clusters into '{args.out}'.")

Saved clusters into 'output/chembl_29_dev/chembl_29_dev_clustering.npy'.


### Investigate X file

In [50]:
print(f"Loading '{args.x}'.")
# X = sc.load_sparse(args.x).tocsr()
X_orig = load_sparse(args.x).tocsr()
X = X_orig[:100, :15000]
print(" X.indptr : " , len(X.indptr), X.indptr)
print(" X.indices: " , len(X.indices) , X.indices)
print(X)
X

In [ ]:
print(f"X shape: {X.shape}")
cl0 = np.arange(X.shape[0])
Xc  = X

In [ ]:
for dist in dists:
    print(f"Running clustering for distance: {dist}")
    cl, cent = cluster(Xc, dist)
    Xc       = Xc[cent]
    cl0      = cl[cl0]




### `clustering()` Line by line

In [ ]:
from libcpp.vector cimport vector
import numpy as np
import scipy.sparse

In [ ]:
X_shape0 = X.shape[0]
X_shape1 = X.shape[1]
x_indptr  = X.indptr
x_indices = X.indices

print(X_shape0, X_shape1, len(x_indptr), len(x_indices))

In [ ]:
## Number of non zero columns 
Xnorm = np.array((X != 0).sum(axis = 1)).flatten()

print(Xnorm.shape)
print(Xnorm.sum())

In [ ]:
dists = np.zeros(X.shape[0], dtype=np.float64)
print(dists.shape)

In [ ]:
clusters = np.zeros(X.shape[0], dtype = int) - 1
perm_ids = np.random.permutation(X.shape[0])       

print(clusters)
print(perm_ids)

In [ ]:

for i in range(X_shape0):
    if i % 10000 == 0:
        print(f"Row {i}.")
    row = perm_ids[i]

    ## computing distances to all centers
    dists[0 : Cnorm.size()] = 0.0
    for j in range(x_indptr[row], x_indptr[row+1]):
        col = x_indices[j]
        for k in range(centers[col].size()):
            dists[ centers[col][k] ] += 1.0

    closest = -1
    min_dist = radius
    for j in range(Cnorm.size()):
        dists[j] = 1.0 - dists[j] / (Xnorm[row] + Cnorm[j] - dists[j])
        if dists[j] < min_dist:
            min_dist = dists[j]
            closest  = j

    if closest >= 0:
        clusters[row] = closest
        continue

    ## create a new cluster
    k = Cnorm.size()
    for j in range(x_indptr[row], x_indptr[row+1]):
        centers[ x_indices[j] ].push_back(k)
    clusters[row] = k
    Cnorm.push_back(Xnorm[row])
    center_ids.push_back(row)

print("Reassigning compounds to the closest clusters.")
num_changed = 0
for row in range(X_shape0):
    if row % 10000 == 0:
        print(f"Row {row}.")
    ## compute distances to all clusters, assign to the closest
    for j in range(x_indptr[row], x_indptr[row+1]):
        col = x_indices[j]
        for k in range(centers[col].size()):
            dists[ centers[col][k] ] += 1.0

    closest = -1
    min_dist = radius + 1e-5
    for j in range(Cnorm.size()):
        tmp_dist = 1.0 - dists[j] / (Xnorm[row] + Cnorm[j] - dists[j])
        if tmp_dist < min_dist:
            min_dist = tmp_dist
            closest  = j
            if min_dist == 0:
                ## best possible
                break
    if (closest >= 0) and (clusters[row] != closest):
        clusters[row]  = closest
        num_changed   += 1

print(f"Reassignement changed {num_changed} assignments.")
print(f"Total {len(center_ids)} clusters.")

return np.asarray(clusters), np.asarray(center_ids)